In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# INTRODUCTION
Les objectifs de la compétition "G-Research Crypto Forecasting" étant très compliqués, nous avons simplifié les données 
pour nous concentrer seulement sur 1 cryptomonnaie et pour la prédire sur 1 mois.
Les résultats ne sont pas vraiment représentatifs du programme car nous l'entrainons sur 
seulement 1 mois de jeu de données.
Si on l'entrainait sur 3 ans, nos prédictions seraient bien plus fiables mais cela prends 
beaucoup trop de temps.
Nous nous sommes principalement aidées d'un tutoriel ainsi que d'autres codes présents 
sur la plateforme.

## Import des librairies

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler # pre-traitement des données
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM # LSTM = Long Short Term Memory

# I. Pre-processing

## Import des données

In [ ]:
train = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv')
assets = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/asset_details.csv')

In [ ]:
display(train.head(10), assets)

## Processing des données

In [ ]:
train["datetime"] =  pd.to_datetime(train['timestamp'], unit='s')
# fusionner les deux df
prices = pd.merge(train, assets, how="outer", on = ["Asset_ID"])

prices.index = prices['timestamp']
prices = prices.sort_index()
ind = prices.index.unique()

In [ ]:
prices

## Prédire les prix du Bitcoin

In [ ]:
# selectionner seulement le bitcoin
btc = prices[prices["Asset_Name"] == "Bitcoin"]
btc_price = btc[["datetime", "Close"]]

# afficher les prix sur l'ensemble du range de temps (i.e. 01-01-2018 au 21-09-2021)
plt.figure(figsize=(12,8))
plt.plot(btc_price["Close"], color="green", label="Bitcoin")
plt.legend()
plt.title("Prix historiques du Bitcoin $")
plt.xlabel('Temps (min)')
plt.ylabel("Price ($)")
plt.show()

In [ ]:
btc_price

# II. Entrainer le modèle

Entrainement sur le mois de janvier 2018 pour ensuite tester le modèle sur les prix du mois de février. Selectionner 1 mois (i.e. 44 640 minutes) pour entrainer le modèle. Début : 1er janvier 2018 à 00h00 / Fin : 31 janvier 2018 à 23h59

Nous utilisons par la suite la mémoire longue à court terme (LSTM) qui est un réseau de neurones complexe utilisé dans le domaine du Deep Learning.
Une unité LSTM commune est composée d'une cellule, d'une porte d'entrée, d'une porte de sortie et d'une porte d'oubli. La cellule se souvient des valeurs sur des intervalles de temps arbitraires et les trois portes régulent le flux d'informations entrant et sortant de la cellule.
Les réseaux LSTM sont bien adaptés à la classification, au traitement et à la réalisation de prédictions basées sur des données de séries chronologiques, car il peut y avoir des décalages de durée inconnue entre des événements importants dans une série chronologique.


In [ ]:
# filtrer nos données sur 01-2018
test_from_date = pd.Timestamp(2018,1,1,0,0) 
test_to_date = pd.Timestamp(2018,1,31,23,59)
btc_price_train = btc_price[btc_price.datetime <= test_to_date] 
print(len(btc_price_train))

# LSTM utilisant tangente hyperbolique (tanh) qui est une mesure sensible à la magnitude des données, nous devons les normaliser les prix en un vecteur compris entre -1 et 1
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(btc_price_train["Close"].values.reshape(-1,1))

# definir l'intervalle de temps à prédire : 15 minutes. 
# nous ne pouvons prédire qu'une seule minute dans dans le futur en se servant des 14 dernières minutes.
prediction_min = 15

In [ ]:
# comme requis pour les réseaux LSTM, nous devons transformer les données d'entrée en (n_échantillons, pas de temps, n_caractéristiques). 
# dans cet exemple, les n_caractéristiques sont 1 et le pas de temps de 14 (données des dernières minutes utilisées pour l'entraînement).

# créer deux listes vides et les remplir avec les données d'entrainement formatées
x_train  = [] # liste de données pour entrainer le modèle
y_train = [] # liste de données que le modèle doit prédire pendant l'entrainement 
for x in range(prediction_min, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_min:x, 0])
    y_train.append(scaled_data[x, 0])
        
x_train, y_train = np.array(x_train), np.array(y_train) # tableau numpy
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# x_train est de dimension (44624, 15, 1). 44624 car nous regardons les 15 minutes passés (44639 - 15 = 44624).
# y_train est seulement de dimension (44624,) car le modèle ne prédit qu'une seule valeur chaque minute.
print('trainX shape == {}.'.format(x_train.shape))
print('trainY shape == {}.'.format(y_train.shape))

In [ ]:
# créer le modèle puis l'entrainer
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2)) #Drop-out est utilisé pour contrôler le sur-ajustement pendant l'entraînement
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1)) # prediction du prix de la prochaine minute
model.compile(optimizer='adam', loss='mean_squared_error')
# fit le modèle
trained = model.fit(x_train, y_train, epochs=15, batch_size=32, validation_split=0.1, verbose=1)

In [ ]:
plt.plot(trained.history['loss'], label='Perte entrainement')
plt.plot(trained.history['val_loss'], label='Perte validation')
plt.legend()

# III. Verifier les performances du modèle

Pour cela, grâce au modèle, nous allons tester les prédictions sur le mois de fevrier 2018.

In [ ]:
# filtrer nos données sur 02-2018
test_from_date = pd.Timestamp(2018,2,2,0,0) 
test_to_date = pd.Timestamp(2018,2,28,23,59) 
test_data = btc_price[(btc_price.datetime >= test_from_date) & (btc_price.datetime <= test_to_date)] 
actual_prices = test_data["Close"].values
total_dataset = pd.concat((btc_price_train["Close"], test_data["Close"]), axis=0)

model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_min:].values
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.transform(model_inputs)

x_test = []
for x in range(prediction_min, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_min:x, 0])
    
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
predicted_prices = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(actual_prices, color="black", label="Actual bitcoin Price")
plt.plot(predicted_prices, color="green", label="Predicted bitcoin Price")
plt.legend()
plt.title("Bitcoin prediction")
plt.xlabel('Temps')
plt.ylabel("Price")

In [ ]:
reel = actual_prices.copy()
pred = np.resize(predicted_prices, predicted_prices.shape[0])        
np.corrcoef(reel, pred)[0,1]

Le modèle semble fonctionner parfaitement sur l'ensemble de données d'entraînement et très bien sur l'ensemble de données de test (correlation de 99%). Cependant, nous devons souligner que le modèle prédit une seule observation à l'avance. Cela signifie que pour chaque nouvelle observation qu'il doit prédire, il prend en entrée les 14 minutes précédentes. Dans la vie réelle, cela est un peu inutile puisque nous voulons principalement prédire de nombreuses observations à l'avance.

# IV. Predisons réellement les 15 prochaines minutes

In [ ]:
real_data = [model_inputs[len(model_inputs) - 100:len(model_inputs), 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))
prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)

x_input = real_data.copy()
temp_input = real_data.copy().reshape(1,-1)
temp_input = list(temp_input)
temp_input = temp_input[0].tolist()
temp_input

lst_output =[]
n_steps=100
i=0
nb_min_pred = 15

while(i<nb_min_pred):
    if(len(temp_input)>100):
        x_input = np.array(temp_input[1:])
        x_input = x_input.reshape(-1,1)
        x_input = x_input.reshape((1, n_steps, 1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        temp_input = temp_input[1:]
        lst_output.extend(yhat.tolist())
        i += 1
    else:
        x_input = x_input.reshape((1, n_steps, 1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        lst_output.extend(yhat.tolist())
        i+=1
print("Done")
output = scaler.inverse_transform(lst_output)
output = np.resize(output, output.shape[0])  

In [ ]:
day_new = np.arange(0, 100)
day_pred = np.arange(85, 100)
reel_data = test_data.Close[len(test_data.Close)-100:len(test_data.Close)]

plt.figure(figsize=(12,8))
plt.plot(day_new, reel_data, color="black", label="Prix actuel Bitcoin")
plt.plot(day_pred, output, color="green", label="Prix prédits Bitcoin")
plt.legend()
plt.title("Bitcoin prediction $")
plt.xlabel('Temps (min)')
plt.ylabel("Price ($)")
plt.show()

# V. Précision de la prédiction (correlation)

In [ ]:
print(np.corrcoef(test_data.Close[len(test_data.Close)-15:len(test_data.Close)].values,output)[0,1])

Le modèle arrive à prédire les 15 prochaines minutes mais semble ne pas être robuste (correlation de 7% environ). Nous pourrions améliorer la précision du modèle en l'entrainant sur un jeu de données plus grand (ici qu'un seul mois), le fit plus longtemps (ici epochs=15), ou bien créer un modèle prenant en compte les volumes tradés ainsi que la corrélation entre les autres actifs.

Il faut aussi prendre en compte le fait qu'il est très compliqué de prévoir les prix qui sont très volatiles. Il n'est donc pas surprenant d'avaoir une corrélation si faible.